In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd
import skforecast

print(skforecast.__version__)

c:\Users\jaesc2\GitHub\skforecast
0.18.0


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.plot.plot import backtesting_gif_creator

In [3]:
y = pd.Series(np.arange(100))
y.index = pd.date_range(start='2022-01-01', periods=100, freq='D')
cv = TimeSeriesFold(
        steps                 = 11,
        initial_train_size    = 70,
        fold_stride           = 5,
        window_size           = 3,
        differentiation       = None,
        refit                 = True,
        fixed_train_size      = True,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
        verbose               = True
    )
folds = cv.split(X=y)
folds

Information of folds
--------------------
Number of observations used for initial training: 70
Number of observations used for backtesting: 30
    Number of folds: 6
    Number skipped folds: 0 
    Number of steps per fold: 11
    Number of steps to exclude between folds (fold stride): 5
    Number of steps to exclude between last observed data (last window) and predictions (gap): 0
    Last fold only includes 5 observations.

Fold: 0
    Training:   2022-01-01 00:00:00 -- 2022-03-11 00:00:00  (n=70)
    Validation: 2022-03-12 00:00:00 -- 2022-03-22 00:00:00  (n=11)
Fold: 1
    Training:   2022-01-06 00:00:00 -- 2022-03-16 00:00:00  (n=70)
    Validation: 2022-03-17 00:00:00 -- 2022-03-27 00:00:00  (n=11)
Fold: 2
    Training:   2022-01-11 00:00:00 -- 2022-03-21 00:00:00  (n=70)
    Validation: 2022-03-22 00:00:00 -- 2022-04-01 00:00:00  (n=11)
Fold: 3
    Training:   2022-01-16 00:00:00 -- 2022-03-26 00:00:00  (n=70)
    Validation: 2022-03-27 00:00:00 -- 2022-04-06 00:00:00  (n=11)


[[[0, 70], [67, 70], [70, 81], [70, 81], True],
 [[5, 75], [72, 75], [75, 86], [75, 86], True],
 [[10, 80], [77, 80], [80, 91], [80, 91], True],
 [[15, 85], [82, 85], [85, 96], [85, 96], True],
 [[20, 90], [87, 90], [90, 100], [90, 100], True],
 [[25, 95], [92, 95], [95, 100], [95, 100], True]]

In [4]:
backtesting_gif_creator(
    data=y,
    cv=cv,
    plot_last_window=False,
    filename="backtesting.gif",
    fps=1
)

Information of folds
--------------------
Number of observations used for initial training: 70
Number of observations used for backtesting: 30
    Number of folds: 6
    Number skipped folds: 0 
    Number of steps per fold: 11
    Number of steps to exclude between folds (fold stride): 5
    Number of steps to exclude between last observed data (last window) and predictions (gap): 0
    Last fold only includes 5 observations.

Fold: 0
    Training:   2022-01-01 00:00:00 -- 2022-03-11 00:00:00  (n=70)
    Validation: 2022-03-12 00:00:00 -- 2022-03-22 00:00:00  (n=11)
Fold: 1
    Training:   2022-01-06 00:00:00 -- 2022-03-16 00:00:00  (n=70)
    Validation: 2022-03-17 00:00:00 -- 2022-03-27 00:00:00  (n=11)
Fold: 2
    Training:   2022-01-11 00:00:00 -- 2022-03-21 00:00:00  (n=70)
    Validation: 2022-03-22 00:00:00 -- 2022-04-01 00:00:00  (n=11)
Fold: 3
    Training:   2022-01-16 00:00:00 -- 2022-03-26 00:00:00  (n=70)
    Validation: 2022-03-27 00:00:00 -- 2022-04-06 00:00:00  (n=11)


'c:\\Users\\jaesc2\\GitHub\\skforecast\\dev\\backtesting.gif'

In [38]:
# Libraries
# ==============================================================================
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from skforecast.datasets import fetch_dataset
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursive
from skforecast.model_selection import TimeSeriesFold
from skforecast.model_selection import backtesting_forecaster
from skforecast.plot import plot_prediction_intervals, set_dark_theme

In [52]:
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "datetime"], "header": 0}
)

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman
/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (204, 2)


In [53]:
# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data[['y']]
data = data.sort_index()
data.head(3)

,y
datetime,
1991-07-01,0.429795
1991-08-01,0.400906
1991-09-01,0.432159


In [54]:
# Create TimeSeriesFold
# ==============================================================================
end_train = '2002-01-01 23:59:00'
cv = TimeSeriesFold(
         steps                 = 10,
         initial_train_size    = len(data.loc[:end_train]),
         window_size           = 10,
         refit                 = True,
         fixed_train_size      = False,
         gap                   = 0,
         allow_incomplete_fold = True,
         verbose               = False
     )

In [69]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor       = LGBMRegressor(random_state=123, verbose=-1),
                 lags            = 15,
                 window_features = RollingFeatures(stats=['mean'], window_sizes=[10])
             )

cv = TimeSeriesFold(
        steps                 = 10,
        initial_train_size    = len(data.loc[:end_train]),
        fold_stride           = 15,
        refit                 = True,
        fixed_train_size      = False,
        gap                   = 0,
        allow_incomplete_fold = True
     )

metric, predictions = backtesting_forecaster(
                          forecaster    = forecaster,
                          y             = data['y'],
                          cv            = cv,
                          metric        = 'mean_squared_error',
                          n_jobs        = 'auto',
                          verbose       = False,
                          show_progress = True
                      )
predictions

  0%|          | 0/6 [00:00<?, ?it/s]

,fold,pred
2002-02-01,0,0.570082
2002-03-01,0,0.731883
2002-04-01,0,0.697942
2002-05-01,0,0.752874
2002-06-01,0,0.730697
2002-07-01,0,0.847688
2002-08-01,0,0.907662
2002-09-01,0,0.932360
2002-10-01,0,0.988048
2002-11-01,0,1.008386


In [66]:
predictions.groupby('fold').size()

fold
0     10
1     10
2     10
3     10
4     10
5     10
6     10
7     10
8     10
9     10
10    10
11    10
12    10
13    10
14     7
15     2
dtype: int64

In [65]:
metric

,mean_squared_error
0,0.008913


In [58]:
preds = [
    pd.Series(np.arange(100)) for i in range(10)
]

In [61]:
fold_labels = [
    np.repeat(i, pred.shape[0]) for i, pred in enumerate(preds)
]
fold_labels = np.concatenate(fold_labels)
fold_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,

In [49]:
folds = 10
steps = 10
fold_stride = 5

preds = []
for fold in range(folds):
    pred = pd.DataFrame(
        {
            'pred': np.arange(steps),
            'fold': [fold] * steps,
        },
        index=pd.date_range(start='2022-01-01', periods=steps, freq='D')
    )
    pred.index = pred.index + DateOffset(days=fold_stride * fold)
    preds.append(pred)

preds = pd.concat(preds)
preds.head(20)

,pred,fold
2022-01-01,0,0
2022-01-02,1,0
2022-01-03,2,0
2022-01-04,3,0
2022-01-05,4,0
2022-01-06,5,0
2022-01-07,6,0
2022-01-08,7,0
2022-01-09,8,0
2022-01-10,9,0


In [51]:
preds.loc[preds.index.duplicated(keep='first')]

,pred,fold
2022-01-06,0,1
2022-01-07,1,1
2022-01-08,2,1
2022-01-09,3,1
2022-01-10,4,1
2022-01-11,0,2
2022-01-12,1,2
2022-01-13,2,2
2022-01-14,3,2
2022-01-15,4,2


In [50]:
preds.loc[~preds.index.duplicated(keep='last')]

,pred,fold
2022-01-01,0,0
2022-01-02,1,0
2022-01-03,2,0
2022-01-04,3,0
2022-01-05,4,0
2022-01-06,0,1
2022-01-07,1,1
2022-01-08,2,1
2022-01-09,3,1
2022-01-10,4,1


In [35]:
%%timeit

preds.loc[~preds.index.duplicated(keep='last')]

95 μs ± 1.44 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [44]:
~preds.index.duplicated(keep='last')

array([ True,  True,  True,  True,  True, False, False, False, False,
       False,  True,  True,  True,  True,  True, False, False, False,
       False, False,  True,  True,  True,  True,  True, False, False,
       False, False, False,  True,  True,  True,  True,  True, False,
       False, False, False, False,  True,  True,  True,  True,  True,
       False, False, False, False, False,  True,  True,  True,  True,
        True, False, False, False, False, False,  True,  True,  True,
        True,  True, False, False, False, False, False,  True,  True,
        True,  True,  True, False, False, False, False, False,  True,
        True,  True,  True,  True, False, False, False, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [42]:
preds.index.duplicated(keep='first')

array([False, False, False, False, False, False, False, False, False,
       False,  True,  True,  True,  True,  True, False, False, False,
       False, False,  True,  True,  True,  True,  True, False, False,
       False, False, False,  True,  True,  True,  True,  True, False,
       False, False, False, False,  True,  True,  True,  True,  True,
       False, False, False, False, False,  True,  True,  True,  True,
        True, False, False, False, False, False,  True,  True,  True,
        True,  True, False, False, False, False, False,  True,  True,
        True,  True,  True, False, False, False, False, False,  True,
        True,  True,  True,  True, False, False, False, False, False,
        True,  True,  True,  True,  True, False, False, False, False,
       False])

In [47]:
preds[preds.duplicated(keep='first')]

,pred,fold


In [37]:
%%timeit

preds.sort_values("fold").loc[~preds.index.duplicated(keep='last')]

176 μs ± 5.08 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
preds.sort_values("fold").drop_duplicates(keep="last")

,pred,fold
2022-01-01,0,0
2022-01-02,1,0
2022-01-02,0,1
2022-01-03,1,1
2022-01-03,0,2
2022-01-04,1,2
2022-01-04,0,3
2022-01-05,1,3
2022-01-05,0,4
2022-01-06,1,4


In [25]:
preds.sort_values("fold").drop_duplicates(keep="last").groupby('fold').size()

fold
0    2
1    2
2    2
3    2
4    2
5    2
6    2
7    2
8    2
9    2
dtype: int64

In [29]:
out = (preds.reset_index(names='date')
           .loc[lambda df: df.groupby('date')["fold"].idxmax()]
           .set_index('date'))
out

,pred,fold
date,,
2022-01-01,0,0
2022-01-02,0,1
2022-01-03,0,2
2022-01-04,0,3
2022-01-05,0,4
2022-01-06,0,5
2022-01-07,0,6
2022-01-08,0,7
2022-01-09,0,8


In [ ]:
pd.testing.assert_frame_equal(
    preds.loc[preds.groupby(level=0)["fold"].idxmax()], 
    preds.loc[~preds.index.duplicated(keep='last')]
)

AssertionError: DataFrame are different

DataFrame shape mismatch
[left]:  (100, 2)
[right]: (55, 2)